In [1]:
"""
    Copyright (c) 2024 Idiap Research Institute, http://www.idiap.ch/
    Written by Cem Bilaloglu <cem.bilaloglu@idiap.ch>

    This file is part of tactileErgodicExploration.

    tactileErgodicExploration is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License version 3 as
    published by the Free Software Foundation.

    tactileErgodicExploration is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with tactileErgodicExploration. If not, see <http://www.gnu.org/licenses/>.
"""

import open3d as o3d
import numpy as np

np.set_printoptions(formatter={"float": lambda x: "{0:0.3e}".format(x)})

import scipy.sparse.linalg as sla

import robust_laplacian
import time
from point_cloud_utils import *


from plotting_utils import *
from virtual_agents import FirstOrderAgent, SecondOrderAgent
import plotly.graph_objects as go
import random

point_cloud_dir = "point_clouds/"

In [2]:
# Parameters
# ==================================================
class param:
    pass  # c-style struct

param.timesteps = 500  # total simulation timesteps

# tuning: [1,100] increasing alpha result in global exploration closer to SS
# decreasing alpha result in local exploration lower limited
param.alpha = 100
# tuning: [25, 300] increasing the value bring more flexibility in selecting
# the pairing alpha. Lowering the value results in faster computation
param.nb_eigen = 200

# eigen corresponds to using Laplacian eigenbasis and using spectral acceleration
# when integrating the diffusion (heat) equation
# exact corresponds to using implicit time stepping for integrating
param.method = "eigen"
# param.method = "exact"

# voxel filter size for downsampling the point cloud
param.voxel_size = 0.003
# radius for the agent footprint that'd be used in coverage
param.agent_radius = 2.5 * param.voxel_size # for the cup and the bunny
# param.agent_radius = 5 * param.voxel_size  # for the plate
# define speed and acceleration in terms of voxel size
param.max_velocity = 1 * param.voxel_size
param.max_acceleration = 0.5 * param.max_velocity

# tuning: doesn't have much effect on exploration so we keep it at 1
param.source_strength = 1

# number of iterations for the non-stationary heat equation
# we kept it at 1 in this work to decrease the computational cost and
# it was enough for the exploration task
param.ndt = 1
# max. num. of neighbors to consider for computing the neighbors in agent radius
param.nb_max_neighbors = 500
# num. of neighbors to consider for tangent space and gradient computation
param.nb_minimum_neighbors = 20
# num. of neighbors to consider for implicitly determining the boundary
# setting this lower in bunny resutls in right ear considered as a seperate body
# setting this higher in bunny results in the right ear being considered as part
# of the main body
param.nb_boundary_neighbors = 40

In [3]:
# Select the object and load the point cloud
# ==========================================
# obj_name = "bun270_X" # Stanford bunny with X projected as the target
# obj_name = "plate_shapes"  # random IKEA plate with hand-drawn shapes
obj_name = "cup_X" # random cup that we scanned with X projected as the target

filename = f"{point_cloud_dir}{obj_name}.ply"
pcloud = process_point_cloud(filename, param)
pcloud.u0 = pcloud.colors[:, 0] # red channel encodes the target

Original Point cloud with 1734 points is downsampled with voxel size 0.003
 resulted in 1511 points
dt: 6.554e-04, h: 2.560e-03, s: 3.000e-03


In [4]:
# Get the discrete Laplacian on the point cloud
# =============================================
"""
Sharp, N., & Crane, K. (2020). A Laplacian for Nonmanifold Triangle Meshes. 
Computer Graphics Forum, 39(5), 69–80. https://doi.org/10.1111/cgf.14069
compute the Laplacian of the point cloud
"""
C, M = robust_laplacian.point_cloud_laplacian(
    pcloud.vertices, n_neighbors=param.nb_boundary_neighbors
)

It is possible to use other methods for solving the heat equation, but here we use Laplacian eigengunctions.

$$L\phi_i=\lambda_iM\phi_i$$

$\lambda_i$ corresponding to first $k$ smallest eigenvalues and $\phi_i$ is the i'th eigenvector , $M$ is the mass matrix and $L$ is weak laplacian then heat equation approximation becomes @sharpDiffusionNetDiscretizationAgnostic2022

$$h_t(u)=\Phi \left[
    {\begin{array}{c}
        e^{-\lambda_1 t}\\
        e^{-\lambda_2 t}\\
		\vdots
    \end{array}}
\right]
\odot (\Phi^TMu)
$$

right side of Hadamard Product is spectral weights, left side of Hadamard Product is projecting back to vertices


In [5]:
# Preprocessing step for solving diffusion
# only needs to be done once the point cloud changes
# ================================================================
if param.method == "exact":
    A_inv = sla.inv(M + pcloud.dt * C)
    A_invM = A_inv @ M
elif param.method == "eigen":
    # Laplacian eigenfunctions
    # compute the eigenvalue decomposition of Laplace-Beltrami
    evals, evecs = sla.eigsh(C, param.nb_eigen, M, sigma=1e-12)
    Phi = evecs
    exp_vector = np.zeros(param.nb_eigen)
    for i in range(param.nb_eigen):
        exp_vector[i] = np.exp(-evals[i] * pcloud.dt)
    first_term = Phi @ exp_vector
    PhiT_M = Phi.T @ M

In [6]:
# Integration step for solving diffusion
# called at each timestep during the runtime
# ================================================================
"""
Spectral acceleration using eigenbasis:
Sharp, N., Attaiki, S., Crane, K., & Ovsjanikov, M. (2022). 
DiffusionNet: Discretization Agnostic Learning on Surfaces. 
ACM Transactions on Graphics, 41(3), 27:1-27:16. https://doi.org/10.1145/3507905
"""
if param.method == "exact":
    ut = A_invM @ pcloud.u0
elif param.method == "eigen":
    ut = pcloud.u0
    second_term = PhiT_M @ ut
    third_term = exp_vector * second_term
    ut = Phi @ third_term

In [7]:
# Click a point on the point cloud to select initial position of the agent
# ==============================================================================
fig = visualize_point_cloud(pcloud.vertices,colors=pcloud.u0,point_size=3,is_show_plot=True)
agent_vertex = None
# create our callback function
def update_point(trace, points,selector):
    global agent_vertex
    for i in points.point_inds:
        agent_vertex = i
        print(f"Initial agent position is set to vertex {agent_vertex}")
    return agent_vertex
fig.data[0].on_click(update_point)
fig

FigureWidget({
    'data': [{'marker': {'color': array([0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00, 0.000e+00]),
                         'colorscale': [[0.0, 'rgb(0,0,255)'], [1.0,
                                        'rgb(255,0,0)']],
                         'opacity': 0.2,
                         'showscale': True,
                         'size': 3},
              'mode': 'markers',
              'name': 'Point Cloud',
              'type': 'scatter3d',
              'uid': '70ade0ff-7164-4932-96cb-167f5e37da51',
              'x': array([3.943e-01, 3.916e-01, 3.971e-01, ..., 3.793e-01, 4.421e-01, 4.413e-01]),
              'y': array([1.916e-01, 1.915e-01, 1.875e-01, ..., 2.536e-01, 1.965e-01, 2.533e-01]),
              'z': array([1.425e-01, 1.419e-01, 1.418e-01, ..., 1.188e-01, 1.064e-01, 1.008e-01])}],
    'layout': {'scene': {'aspectmode': 'data',
                         'camera': {'center': {'x': 0, 'y': 0, 'z': 0},
                                    'e

In [8]:
#  Initialize the agent and visualize the initial position on the point cloud
# ===============================
agent = FirstOrderAgent(
    x=np.zeros(3), dim_t=param.timesteps, max_velocity=param.max_velocity
)

agent = SecondOrderAgent(
    x=np.zeros(3), dim_t=param.timesteps, max_velocity=param.max_velocity, max_acceleration=param.max_acceleration
)
agent.radius = param.agent_radius
if agent_vertex is None:
    # if no initial position selected randomly select the vertex 
    agent_vertex = random.sample(range(0, pcloud.vertices.shape[0]), 1)[0]

agent.x = pcloud.vertices[agent_vertex,:] # set the initial position to the selected vertex
print(f"Agent initial position: {agent.x}")
plots = []
# visualize the agents' initial position
x = agent.x
scatter = go.Scatter3d(
    x=[x[0]],
    y=[x[1]],
    z=[x[2]],
    name="Agent",
    marker=dict(
        size=8,
        color="rgb(0,255,0)",
    ),
    line=dict(width=0.1),
)
plots.append(scatter)
visualize_point_cloud(pcloud.vertices, colors=pcloud.u0, plots=plots, is_show_plot=True)

Agent initial position: [3.641e-01 1.875e-01 1.250e-01]


FigureWidget({
    'data': [{'line': {'width': 0.1},
              'marker': {'color': 'rgb(0,255,0)', 'size': 8},
              'name': 'Agent',
              'type': 'scatter3d',
              'uid': '19a15d4d-f759-468a-8d98-3edef38b7d5b',
              'x': [0.36407533288002014],
              'y': [0.18745116889476776],
              'z': [0.125043123960495]},
             {'marker': {'color': array([0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00, 0.000e+00]),
                         'colorscale': [[0.0, 'rgb(0,0,255)'], [1.0,
                                        'rgb(255,0,0)']],
                         'opacity': 0.2,
                         'showscale': True,
                         'size': 3},
              'mode': 'markers',
              'name': 'Point Cloud',
              'type': 'scatter3d',
              'uid': '7e2629fc-6616-4d3b-b347-4ac2e920d87b',
              'x': array([3.943e-01, 3.916e-01, 3.971e-01, ..., 3.793e-01, 4.421e-01, 4.413e-01]),
     

In [9]:
def hedac(agent, param, pcloud):
    """
    Perform HEDAC exploration using the agent and the point cloud.

    Original implementation on a 2-D rectangular grid by Ivic et al.
    Ivić, S., Crnković, B., & Mezić, I. (2017). Ergodicity-Based Cooperative 
    Multiagent Area Coverage via a Potential Field. IEEE Transactions on 
    Cybernetics, 47(8), 1983–1993. https://doi.org/10.1109/TCYB.2016.2634400

    Args:
        agent (Agent): The agent object representing the virtual exploration agent.
        param (Parameters): The parameters for the HEDAC algorithm.
        pcloud (PointCloud): The point cloud object representing the exploration
             domain and target

    Returns:
        tuple: A tuple containing the agent's trajectory, heat array,
            coverage array, and time array.
    """
    coverage_arr = np.zeros((len(pcloud.vertices), param.timesteps))
    heat_arr = np.zeros_like(coverage_arr)

    # we normalize the goal because it should be a probability distribution
    goal_density = pcloud.u0/ (np.sum(pcloud.u0) + 1e-12)

    # we keep this and add coverage at each timestep on top of it
    coverage = np.zeros_like(goal_density)
    ut = np.array(goal_density)

    agent.t = 0  # reset the agent's time
    # do absolute minimum inside the main loop
    for t in range(param.timesteps):
        dists, neighbor_ids, neighbor_coords = get_pcloud_neighbors(
            pcloud.pcd_tree,
            pcloud.vertices,
            np.copy(agent.x),
            param.nb_max_neighbors,
            param.nb_minimum_neighbors,
            agent.radius
        )

        # Compute the coverage using RBF kernel
        kernel_vals = np.exp(-(1 / agent.radius) * dists**2)
        coverage[neighbor_ids] += kernel_vals

        neighbor_ids = neighbor_ids[: param.nb_minimum_neighbors]
        dists = dists[: param.nb_minimum_neighbors]
        neighbor_coords = pcloud.vertices[neighbor_ids, :]

        # Consider the coverage as a probability distribution
        coverage_density = coverage/ (np.sum(coverage) + 1e-12)
        source = np.maximum(goal_density - coverage_density, 0) ** 2
        source = source / (np.sum(source) + 1e-12)

        # source *= pcloud.surface_area

        if param.method == "eigen":
            second_term = PhiT_M @ ut
            third_term = exp_vector * second_term
            ut = Phi @ third_term
        elif param.method == "exact":
            ut = A_inv @ M @ ut


        ut += param.source_strength * source
        # Add the source term
        (
            agent.x,
            gradient,
            projected_neighbor_coords,
        ) = get_gradient(
            np.copy(agent.x),
            neighbor_coords,
            neighbor_ids,
            ut,
        )
        # weight the gradient by the heat at the agent's position
        gradient = gradient * ut[neighbor_ids[0]]
        agent.update(gradient)

        coverage_arr[..., t] = coverage
        heat_arr[..., t] = np.copy(ut)
    return agent.x_arr, heat_arr, coverage_arr


def compute_coverage_residual(initial_heat_arr, coverage_arr):
    """
    Compute the normalized residual between the goal density and the
    coverage density.

    Args:
        initial_heat_arr (numpy.ndarray): Array of initial heat maps representing the goal density.
        coverage_arr (numpy.ndarray): Array of heat maps representing the coverage density.

    Returns:
        numpy.ndarray: Array of normalized residuals.
        numpy.ndarray: Array of goal densities.
    """
    goal_density = initial_heat_arr/ (np.sum(initial_heat_arr) + 1e-12)
    # [:,None] is required to match the dimensions for broadcasting
    residual_arr = goal_density[:, None] - coverage_arr
    uncovered_arr = np.where(residual_arr < 0, 0, residual_arr)
    # overcovered_arr = np.where(residual_arr > 0, 0, residual_arr)
    normalized_residual = np.linalg.norm(uncovered_arr, axis=0) / np.linalg.norm(
        goal_density
    )

    return normalized_residual

In [10]:
# Run and visualize an experiment
# =================================================
x_arr, heat_arr, coverage_arr = hedac(agent, param, pcloud)
plots = visualize_point_cloud(
    pcloud.vertices, colors=pcloud.u0, is_show_plot=False, point_size=3
)
fig = visualize_trajectory(x_arr[:,:], plots)

fig.show()

# Compute and plot the coverage error
# ==========================
normalized_residual = compute_coverage_residual(heat_arr[..., 0], coverage_arr)

# plot the coverage error
fig = go.Figure()
# add traces for each error
fig.add_trace(
    go.Scatter(y=normalized_residual, mode="lines", name=f"Experiment {i}")
)
fig.update_layout(
    xaxis_title="Timesteps",
    yaxis_title="Normalized Ergodic Metric",
    title="Normalized Coverage Residual",
)
fig.show()